In [1]:
library(DESeq2)
library(RColorBrewer)
library(reshape2)
library("gplots")
library("GGally")                      # Load GGally package
library(goseq)
library(readxl)
library(tidyverse)
library("pheatmap")
library(corrplot)
library(circlize)

Warning message:
"package 'DESeq2' was built under R version 4.2.2"
Loading required package: S4Vectors

Warning message:
"package 'S4Vectors' was built under R version 4.2.2"
Loading required package: stats4

Loading required package: BiocGenerics

Warning message:
"package 'BiocGenerics' was built under R version 4.2.1"

Attaching package: 'BiocGenerics'


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: 'S4Vectors'


The following objects are masked from 'package:base':

    exp

In [2]:
library(svglite)

Warning message:
"package 'svglite' was built under R version 4.2.3"


In [3]:
library(ComplexUpset)

Warning message:
"package 'ComplexUpset' was built under R version 4.2.2"


In [4]:
library(ggrepel)

Warning message:
"package 'ggrepel' was built under R version 4.2.1"


In [5]:
library(ComplexHeatmap)

Warning message:
"package 'ComplexHeatmap' was built under R version 4.2.1"
Loading required package: grid

ComplexHeatmap version 2.14.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite either one:
- Gu, Z. Complex Heatmap Visualization. iMeta 2022.
- Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
    genomic data. Bioinformatics 2016.


The new InteractiveComplexHeatmap package can directly export static 
complex heatmaps into an interactive Shiny app with zero effort. Have a try!

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))
! pheatmap() has been masked by ComplexHeatmap::pheatmap(). Most of the arguments
   in the original pheatmap() are identically supported in the new function. You 
   can still use the original fu

In [6]:
source('../src/common_functions.R')

In [7]:
options(repr.plot.width = 10, repr.plot.height = 10, repr.plot.res = 300)

# Enrichment of differentially expressed proteins and transcripts

Compare DE genes and DE proteins and run enrichment for all

# Create DE df

# GOSEQ

In [8]:
df_transport_classes = read_csv(file.path('DE_results', 'transport_classes.csv'), name_repair = 'universal' )

New names:
• `Original Name` -> `Original.Name`
• `Compound Category` -> `Compound.Category`
• `Substrate(s)` -> `Substrate.s.`
Rows: 48 Columns: 6
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (6): Original.Name, Type, Compound.Category, Substrate.s., Direction, Notes

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [9]:
df_all = read_csv(file.path('DE_results', 'HOT1A3_all_FC_results.csv'))

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details, e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 101690 Columns: 36
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (24): contrast, protein_id, Assay, contrast1, seq_id, Name, gene, locus_...
dbl  (7): logFC, AveExpr, pvalue, padj, start, end, gene_length
lgl  (5): gene_synonym, exception, down, up, is_de

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [10]:
goseq_res = read_csv(file.path('DE_results', 'HOT1A3_GOSEQ_results.csv'))

Rows: 10396 Columns: 17
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (12): category, type, enrich, contrast, organism, main, sub, module, Cat...
dbl  (5): over_represented_pvalue, padj, under_represented_pvalue, numDEInCa...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [11]:
goseq_res = goseq_res %>%
    mutate(Category = factor(Category, levels=category.order))


In [12]:
head(goseq_res)

category,over_represented_pvalue,padj,under_represented_pvalue,numDEInCat,numInCat,type,enrich,contrast,organism,main,sub,module,Category,PATH,pathway,path
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<chr>,<chr>
09122,2.962018e-11,7.997449e-10,1.0000000,25,76,down,sub,C2vsC1_prot,HOT1A3,09120 Genetic Information Processing,09122 Translation,Translation,Genetic Info,09122 Translation,Translation,NA
09182,2.954313e-04,3.988323e-03,0.9998623,47,380,down,sub,C2vsC1_prot,HOT1A3,09180 Brite Hierarchies,09182 Protein families: genetic information processing,Protein families: genetic information processing,Genetic Info,09182 Protein families: genetic information processing,Protein families: genetic information processing,NA
09105,7.756781e-04,6.981103e-03,0.9996844,26,170,down,sub,C2vsC1_prot,HOT1A3,09100 Metabolism,09105 Amino acid metabolism,Amino acid,AA/Nucleotide,09105 Amino acid metabolism,Amino acid metabolism,NA
09102,4.319859e-03,2.334114e-02,0.9983626,16,96,down,sub,C2vsC1_prot,HOT1A3,09100 Metabolism,09102 Energy metabolism,Energy,Energy/Carbohydrate/Glycan,09102 Energy metabolism,Energy metabolism,NA
09142,4.322433e-03,2.334114e-02,0.9987031,11,54,down,sub,C2vsC1_prot,HOT1A3,09140 Cellular Processes,09142 Cell motility,Cell motility,Motility,09142 Cell motility,Cell motility,NA
09132,3.774030e-02,1.698314e-01,0.9821703,14,99,down,sub,C2vsC1_prot,HOT1A3,09130 Environmental Information Processing,09132 Signal transduction,Signal transduction,Env. Info/Cellular Process,09132 Signal transduction,Signal transduction,NA


In [13]:
goseq_meta = goseq_res %>% distinct(contrast) %>% 
    separate_wider_delim(contrast, "_", names = c(NA, "Assay"), cols_remove = FALSE) %>%
    mutate(Group = case_when(
        str_detect(contrast, "C")~ "Coculture", 
        str_detect(contrast, "P3|P5|LATEvsP")~ "Axenic late", 
        TRUE~ "Axenic"
    )) %>% 
    mutate(
        Group = factor(Group, levels=c("Coculture", "Axenic", "Axenic late")),
        Assay = factor(Assay, levels=c("rna", "prot"))
          ) %>%
    arrange(Group, Assay, contrast) %>%
    column_to_rownames('contrast') 
goseq_meta 

,Assay,Group
,<fct>,<fct>
C1vsA1_rna,rna,Coculture
C2vsC1_rna,rna,Coculture
C3vsC1_rna,rna,Coculture
C3vsC2_rna,rna,Coculture
C4vsC1_rna,rna,Coculture
C4vsC3_rna,rna,Coculture
C5vsC1_rna,rna,Coculture
C5vsC4_rna,rna,Coculture
LATEvsC1_rna,rna,Coculture


In [14]:
options(repr.plot.width = 15, repr.plot.height = 15, repr.plot.res = 300)

In [15]:
goseq_pal_fc_fun = colorRamp2(c(-5,-2,-1, 0, 1,2, 5), rev(RColorBrewer::brewer.pal(7, "BrBG")))
pfcgoseq = goseq_pal_fc_fun(seq(-10, 10, length.out = 20))
#pfcgoseq = goseq_pal_up_fun(c(-5, -2, 0, 2, 5))
#show_col(pfcgoseq)

In [16]:
group_pal = c(Coculture='darkgreen', Axenic='#A6BB8D','Axenic late'='darkred')


In [17]:
coclture_assays = goseq_meta %>% filter(Group == "Coculture") %>% row.names()

In [18]:
contrast_list_longterm = c(
    'LATEvsC1_prot', 
    'LATEvsC1_rna',
    'A3vsA1_prot', 
    'A3vsA1_rna'
    
)

In [19]:
dput(sort(unique(goseq_res$contrast)))
contrast_list = c( "LATEvsC1_prot", #"LATEvsC2_prot",  
"A2vsA1_prot",  #"LATEvsP1_prot", 
"LATEvsP2_prot", #"LATEvsP1_rna", 
                  "LATEvsP2_rna", "LATEvsC1_rna", "LATEvsP1_prot",
#"LATEvsC2_rna",  
                  "P2vsP1_rna" )

contrast_list_survival = c( 
    "LATEvsC1_prot", #"LATEvsC2_prot",  
    "LATEvsC1_rna", 
"P2vsP1_prot",  #"LATEvsP1_prot", 
                  "P2vsP1_rna" 
 #"LATEvsP1_rna", 
                  #"LATEvsP2_rna", 
#"LATEvsC2_rna",  
)
contrast_list_death = c( 
     
"P2vsP1_prot",  "LATEvsP1_prot", 
                  "P2vsP1_rna" ,
"LATEvsP2_prot"
)

contrast_list_coculture = c( 
    "C2vsC1_prot" , 
    "C2vsC1_rna" , 
    "C3vsC2_prot", 
    "C3vsC2_rna" , 
    "C4vsC3_prot", 
    "C4vsC3_rna" , 
    "C5vsC4_prot", 
    "C5vsC4_rna" 
    )

comparison_order_survival = c("Death\nvs growth", "Survival\nvs growth")
comparison_order_death = c("Death\nvs growth", "Decompsing\nvs death", "Decompsing\nvs growth" )

c("A2vsA1_prot", "A2vsA1_rna", "A3vsA1_prot", "A3vsA1_rna", "A3vsA2_prot", 
"A3vsA2_rna", "A5vsA1_prot", "A5vsA3_prot", "C1vsA1_rna", "C2vsC1_prot", 
"C2vsC1_rna", "C3vsC1_prot", "C3vsC1_rna", "C3vsC2_prot", "C3vsC2_rna", 
"C4vsC1_prot", "C4vsC1_rna", "C4vsC3_prot", "C4vsC3_rna", "C5vsC1_prot", 
"C5vsC1_rna", "C5vsC4_prot", "C5vsC4_rna", "LATEvsA1_rna", "LATEvsA2_rna", 
"LATEvsA3_rna", "LATEvsC1_prot", "LATEvsC1_rna", "LATEvsC2_prot", 
"LATEvsC2_rna", "LATEvsC3_prot", "LATEvsC3_rna")


In [20]:
contrast_list_pro_continues_minus_C3_c1 = c(
    "P2vsP1_prot", 
    "P3vsP1_prot", 
    "P5vsP1_prot",
    "P2vsP1_rna", 

    "C2vsC1_prot", 
   # "C3vsC1_prot", 
    "C4vsC1_prot", 
    "C5vsC1_prot",
	
    "C2vsC1_rna", 
    "C3vsC1_rna", 
    "C4vsC1_rna", 
    "C5vsC1_rna", 
	
    "C3vsC2_prot", 
    "C4vsC3_prot", 
    "C5vsC4_prot", 
	
    "C3vsC2_rna", 
    "C4vsC3_rna", 
    "C5vsC4_rna"
	
)


In [21]:
-log10(0.05)

[1] 1.30103

In [22]:
names(df_all)

[1] "contrast"           "protein_id"         "logFC"             
 [4] "AveExpr"            "pvalue"             "padj"              
 [7] "Assay"              "contrast1"          "seq_id"            
[10] "Name"               "gene"               "gene_synonym"      
[13] "locus_tag"          "old_locus_tag"      "source"            
[16] "start"              "end"                "strand"            
[19] "Note"               "exception"          "inference"         
[22] "product"            "gene_length"        "path_id"           
[25] "main"               "sub"                "path"              
[28] "ecpath"             "Category"           "Membrane.transport"
[31] "kegg_gene1"         "kegg_gene2"         "gene_id"           
[34] "down"               "up"                 "is_de"

In [23]:
genome = 'HOT1A3'

In [31]:
deg = df_all %>% filter(padj < 0.05) %>% distinct(gene_id)
df_all %>% 
    filter(Membrane.transport %in% 'Membrane transport') %>%
    #filter(gene_id %in% deg$gene_id) %>% 
    distinct(gene_id, .keep_all = TRUE) %>%
    count(path) %>% arrange(desc(n))


path,n
<chr>,<int>
02000 Transporters [BR:ko02000],204
02000 Transporters [BR:ko02000];02010 ABC transporters [PATH:ko02010],41
99977 Transport,20
02044 Secretion system [BR:ko02044];03070 Bacterial secretion system [PATH:ko03070];05111 Biofilm formation - Vibrio cholerae [PATH:ko05111],11
02024 Quorum sensing [PATH:ko02024];02044 Secretion system [BR:ko02044];03060 Protein export [PATH:ko03060];03070 Bacterial secretion system [PATH:ko03070],8
00480 Glutathione metabolism [PATH:ko00480];00980 Metabolism of xenobiotics by cytochrome P450 [PATH:ko00980];00982 Drug metabolism - cytochrome P450 [PATH:ko00982];00983 Drug metabolism - other enzymes [PATH:ko00983];02000 Transporters [BR:ko02000],5
02000 Transporters [BR:ko02000];02020 Two-component system [PATH:ko02020],5
02000 Transporters [BR:ko02000];02024 Quorum sensing [PATH:ko02024],5
02044 Secretion system [BR:ko02044];03060 Protein export [PATH:ko03060];03070 Bacterial secretion system [PATH:ko03070],5


In [32]:
deg = df_all %>% filter(padj < 0.05) %>% distinct(gene_id)
df_all %>% 
    filter(Membrane.transport %in% 'Membrane transport') %>%
    #filter(gene_id %in% deg$gene_id) %>% 
    distinct(gene_id, .keep_all = TRUE) %>%
    count(ecpath) %>% arrange(desc(n))


ecpath,n
<chr>,<int>
K02014 TC.FEV.OM; iron complex outermembrane recepter protein,35
"K15727 czcB, cusB, cnrB; membrane fusion protein, heavy metal efflux system",7
K03561 exbB; biopolymer transport protein ExbB,6
"K00799 GST, gst; glutathione S-transferase [EC:2.5.1.18]",5
K01992 ABC-2.P; ABC-2 type transport system permease protein,5
"K03307 TC.SSS; solute:Na+ symporter, SSS family",5
K03442 mscS; small conductance mechanosensitive channel,5
K02030 ABC.PA.S; polar amino acid transport system substrate-binding protein,4
K03559 exbD; biopolymer transport protein ExbD,4


In [25]:
transport_genes = df_all %>% 
    filter(Membrane.transport %in% 'Membrane transport') %>%
    #filter(padj < 0.05) %>% 
    distinct(gene_id) 
transport_de_genes = df_all %>% 
    filter(Membrane.transport %in% 'Membrane transport') %>%
    filter(padj < 0.05) %>% 
    distinct(gene_id) 
df_transport = df_all %>% 
    #filter(Membrane.transport %in% 'Membrane transport') %>%
    filter(gene_id %in% transport_genes$gene_id) %>% 
    distinct(gene_id, .keep_all = TRUE) %>%
    extract(ecpath, c("system"), "^K[[:digit:]]+ *[^;]*; ([^\\[,]+)", remove = FALSE) %>%
    mutate(system = if_else(str_detect(system, "system"),str_remove(system, "system.*"), system)) %>%
    mutate(system = if_else(str_detect(system, "subunit"),str_remove(system, "subunit.*"), system)) %>%
    mutate(system = if_else(str_detect(system, " protein"),str_remove(system, " protein.*"), system)) %>% # add the space to avoid preprotein
    #extract(ecpath, c("system"), "^K[[:digit:]]+ *[^;]*; ([^\\[]+)", remove = FALSE) %>%
    mutate(system=str_trim(system)) #%>%
    #count(system) %>% write_csv('tmp.csv')


In [27]:
df_transport %>%count(system) %>% write_csv('tmp.csv')

In [ ]:
names(df_transport)
names(df_transport_classes)

In [ ]:

gene_meta = df_transport %>% 
    distinct(gene_id, .keep_all = TRUE) %>% 
    mutate(protein.export = if_else(str_detect(path, regex('Protein export', ignore_case=TRUE)), 'Protein export', '')) %>%
    mutate(secretion.system = if_else(str_detect(path, regex('Secretion system', ignore_case=TRUE)), 'Secretion system', '')) %>%
    left_join((df_transport_classes %>% rename(system=Original.Name)), by='system') %>%
    mutate(Direction = if_else(Direction == 'bidirectional', 'other', Direction)) %>%
    mutate(Compound.Category = if_else(protein.export == 'Protein export', 'Protein', Compound.Category)) %>%
    mutate(Direction = if_else(protein.export == 'Protein export', 'export', Direction)) %>%
    distinct(gene_id, gene, system, Compound.Category,Direction, Substrate.s.) %>%
    column_to_rownames('gene_id')
head(gene_meta)


#    distinct(gene_id, gene, system, sub,path,ecpath, protein.export, secretion.system) %>%
    #count(Compound.Category,Direction,  is_de) %>% pivot_wider(names_from = 'is_de', values_from = 'n', values_fill = 0) 

## Transporters

In [ ]:
module_name = 'Membrane transport'
module_title = paste0(genome, ' ',module_name)

In [ ]:

degenes = df_all %>% 
    filter(Membrane.transport %in% 'Membrane transport') %>%
    filter(padj < 0.05) %>% 
    filter(contrast1 %in% contrast_list_survival) %>%
    distinct(gene_id) 

degenes

In [ ]:
df_all %>% filter(gene_id %in% df_transport$gene_id) %>% 
    filter(contrast1 %in% contrast_list_survival) %>%
    replace_na(list(padj=1)) %>%
    mutate(star = if_else(padj < 0.05, '*', '')) %>%
    pivot_wider(id_cols = contrast1, values_from = star, names_from = gene_id, values_fill = '') %>% 
    mutate(contrast1 = factor(contrast1, levels=contrast_list)) %>%
    arrange(contrast1) %>%
    column_to_rownames('contrast1')  %>%
    relocate(rownames(gene_meta)) 

In [ ]:
gene_stars_survival = df_all %>% filter(gene_id %in% df_transport$gene_id) %>% 
    filter(contrast1 %in% contrast_list_survival) %>%
    replace_na(list(padj=1)) %>%
    mutate(star = if_else(padj < 0.05, '*', '')) %>%
    pivot_wider(id_cols = contrast1, values_from = star, names_from = gene_id, values_fill = '') %>% 
    mutate(contrast1 = factor(contrast1, levels=contrast_list_survival)) %>%
    arrange(contrast1) %>%
    column_to_rownames('contrast1')  %>%
    relocate(rownames(gene_meta)) 


In [ ]:
contrast_map_to_label_pro_survival_death[contrast_list_survival]

In [ ]:

gene_mat = df_all %>% filter(gene_id %in% df_transport$gene_id) %>% 
    filter(contrast1 %in% contrast_list_survival) %>%
    pivot_wider(id_cols = contrast1, values_from = logFC, names_from = gene_id) %>% 
    mutate(contrast1 = factor(contrast1, levels=contrast_list_survival)) %>%
    arrange(contrast1) %>%
    column_to_rownames('contrast1')  %>%
    relocate(rownames(gene_meta)) %>% as.matrix() %>%
Heatmap(
     name = 'logFC',
    col = goseq_pal_fc_fun,
#     rect_gp = gpar(col = "white", lwd = 2),
     cluster_rows  = FALSE,
     show_column_dend = FALSE,
     #row_order = rownames(goseq_meta),
     row_split = contrast_map_to_label_pro_survival_death[contrast_list_survival],
    
#     #row_title_gp = gpar(col = group_pal),
#     row_names_gp = gpar(col = group_pal),
     border=TRUE,
#     width=unit(1, 'cm'),
     height=unit(6, 'cm'),
     heatmap_legend_param = list(direction = "horizontal"),
     row_title=NULL,
     #column_title=NULL,
    column_split = gene_meta %>% select(Compound.Category, Direction), # Substrate.s.),
    column_title_gp = gpar(fontsize = 10),    
    column_title_rot = 90,
    column_labels=gene_meta$gene,
#     top_annotation = HeatmapAnnotation(
#         Pathway=gene_meta$system, 
#         #show_legend =FALSE,
#         #col = list(Pathway = c('Yes' = "grey", 'No' = "white"))#,
#         #show_legend=FALSE,
#     ),
    
    cell_fun = function(j, i, x, y, width, height, fill) {
        grid.text(sprintf("%s", gene_stars_survival[i, j]), x, y, gp = gpar(fontsize = 9))
    },
    
    #column_title = 'MED4: GOSEQ Score - upregulated',
) #, annotation_col = goseq_meta) #, annotation_row = path_meta)
ht_list = gene_mat  #+ ann_mat
draw(ht_list, heatmap_legend_side = "bottom", merge_legend = TRUE,column_title = module_title )

In [ ]:
gene_stars_continues = df_all %>% filter(gene_id %in% df_transport$gene_id) %>% 
    filter(contrast1 %in% contrast_list_pro_continues) %>%
    replace_na(list(padj=1)) %>%
    mutate(star = if_else(padj < 0.05, '*', '')) %>%
    pivot_wider(id_cols = contrast1, values_from = star, names_from = gene_id, values_fill = '') %>% 
    mutate(contrast1 = factor(contrast1, levels=contrast_list_pro_continues)) %>%
    arrange(contrast1) %>%
    column_to_rownames('contrast1')  %>%
    relocate(rownames(gene_meta)) 


In [ ]:

gene_mat = df_all %>% filter(gene_id %in% df_transport$gene_id) %>% 
    filter(contrast1 %in% contrast_list_pro_continues) %>%
    pivot_wider(id_cols = contrast1, values_from = logFC, names_from = gene_id) %>% 
    mutate(contrast1 = factor(contrast1, levels=contrast_list_pro_continues)) %>%
    arrange(contrast1) %>%
    column_to_rownames('contrast1')  %>%
    relocate(rownames(gene_meta)) %>% as.matrix() %>%
Heatmap(
     name = 'logFC',
    col = goseq_pal_fc_fun,
#     rect_gp = gpar(col = "white", lwd = 2),
     cluster_rows  = FALSE,
     show_column_dend = FALSE,
     #row_order = rownames(goseq_meta),
     row_split = contrast_map_to_label_pro_continues[contrast_list_pro_continues],
    
#     #row_title_gp = gpar(col = group_pal),
#     row_names_gp = gpar(col = group_pal),
     border=TRUE,
#     width=unit(1, 'cm'),
     height=unit(6, 'cm'),
     heatmap_legend_param = list(direction = "horizontal"),
     row_title=NULL,
     #column_title=NULL,
    column_split = gene_meta %>% select(Compound.Category, Direction), #, Substrate.s.),
    column_title_gp = gpar(fontsize = 10),    
    column_title_rot = 90,
    column_labels=gene_meta$gene,
#     top_annotation = HeatmapAnnotation(
#         Pathway=gene_meta$system, 
#         #show_legend =FALSE,
#         #col = list(Pathway = c('Yes' = "grey", 'No' = "white"))#,
#         #show_legend=FALSE,
#     ),
    
    cell_fun = function(j, i, x, y, width, height, fill) {
        grid.text(sprintf("%s", gene_stars_continues[i, j]), x, y, gp = gpar(fontsize = 9))
    },
    
    #column_title = 'MED4: GOSEQ Score - upregulated',
) #, annotation_col = goseq_meta) #, annotation_row = path_meta)
ht_list = gene_mat  #+ ann_mat
draw(ht_list, heatmap_legend_side = "bottom", merge_legend = TRUE,column_title = module_title )

In [ ]:
paste0(unique(gene_meta$system), collapse = ' , ')

In [ ]:
sessionInfo()

In [ ]:
help(str_detect)